In [2]:
import cv2
from ultralytics import YOLO
import time

# ============================================
# REAL-TIME OBJECT DETECTION WITH YOLOv11
# ============================================

def main():
    """
    Main function for object detection using YOLOv11
    """
    
    print("=" * 60)
    print("YOLOv11 OBJECT DETECTION SYSTEM")
    print("=" * 60)
    
    # ========== CONFIGURATION ==========
    VIDEO_PATH = r"PATH"
    CONFIDENCE_THRESHOLD = 0.5
    MODEL_NAME = "yolo11n.pt"
    
    # ========== TARGET OBJECTS TO DETECT ==========
    # Note: "projector" and "air conditioner" are not in standard YOLO COCO dataset
    # They won't be detected unless you use a custom trained model
    TARGET_ITEMS = {
        "laptop": ("Laptop", (0, 255, 0)),          # Green
        "tv": ("Monitor", (255, 0, 0)),             # Blue
        "keyboard": ("Keyboard", (0, 255, 255)),    # Yellow
        "chair": ("Chair", (128, 0, 128)),          # Purple
        # These will only work with custom trained model:
        "projector": ("Projector", (255, 165, 0)),  # Orange
        "air conditioner": ("AC", (0, 0, 255)),     # Red
    }
    
    print("\n[INFO] Loading YOLOv11 model...")
    
    # ========== LOAD YOLOv11 MODEL ==========
    try:
        model = YOLO(MODEL_NAME)
        print(f"[SUCCESS] Model loaded: {MODEL_NAME}")
        
        # Check which target items are available in the model
        available_items = []
        unavailable_items = []
        
        for item in TARGET_ITEMS.keys():
            if item in model.names.values():
                available_items.append(item)
            else:
                unavailable_items.append(item)
        
        print(f"[INFO] Available items in model: {len(available_items)}/{len(TARGET_ITEMS)}")
        if available_items:
            print(f"  Available: {', '.join(available_items)}")
        if unavailable_items:
            print(f"  Not available (need custom training): {', '.join(unavailable_items)}")
            
    except Exception as e:
        print(f"[ERROR] Failed to load model: {e}")
        return
    
    # ========== OPEN VIDEO SOURCE ==========
    print("\n[INFO] Opening video source...")
    
    cap = cv2.VideoCapture(VIDEO_PATH)
    
    if not cap.isOpened():
        print(f"[ERROR] Cannot open video: {VIDEO_PATH}")
        print("[INFO] Trying webcam...")
        cap = cv2.VideoCapture(0)  # Fallback to webcam
    
    if not cap.isOpened():
        print("[ERROR] Cannot open any video source!")
        return
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"[INFO] Video Properties:")
    print(f"  • Resolution: {width}x{height}")
    print(f"  • FPS: {fps}")
    
    # ========== INITIALIZE COUNTERS ==========
    max_counts = {display_name: 0 for display_name, _ in TARGET_ITEMS.values()}
    frame_count = 0
    start_time = time.time()
    
    print("\n[INFO] Starting detection...")
    print("[CONTROLS] Press 'Q' to quit | Press 'R' to reset counters")
    print("[CONTROLS] Press 'S' to save screenshot")
    print("=" * 60)
    
    # ========== MAIN DETECTION LOOP ==========
    while True:
        ret, frame = cap.read()
        if not ret:
            print("\n[INFO] End of video reached")
            break
        
        frame_count += 1
        height, width = frame.shape[:2]
        
        # ===== RUN YOLOv11 DETECTION =====
        results = model(frame, conf=CONFIDENCE_THRESHOLD, verbose=False)
        
        # ===== RESET COUNTERS FOR CURRENT FRAME =====
        current_counts = {display_name: 0 for display_name, _ in TARGET_ITEMS.values()}
        
        # ===== PROCESS DETECTIONS =====
        for result in results:
            if result.boxes is not None:
                for box in result.boxes:
                    # Extract box information
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    confidence = float(box.conf[0])
                    class_id = int(box.cls[0])
                    class_name = model.names[class_id]
                    
                    # Default color for non-target objects
                    color = (200, 200, 200)
                    label = class_name
                    
                    # Check if this is a target object
                    if class_name in TARGET_ITEMS:
                        display_name, color = TARGET_ITEMS[class_name]
                        label = f"{display_name} {confidence:.2f}"
                        current_counts[display_name] += 1
                    else:
                        label = f"{class_name} {confidence:.2f}"
                    
                    # Draw bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    
                    # Draw label with background
                    (text_width, text_height), _ = cv2.getTextSize(
                        label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1
                    )
                    
                    # Draw background rectangle for text
                    cv2.rectangle(frame, 
                                 (x1, y1 - text_height - 5),
                                 (x1 + text_width, y1 - 5),
                                 color, -1)
                    
                    # Draw text
                    cv2.putText(frame, label, (x1, y1 - 10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                               (255, 255, 255), 1)
        
        # ===== UPDATE MAXIMUM COUNTS =====
        for item in max_counts:
            max_counts[item] = max(max_counts[item], current_counts[item])
        
        # ===== DISPLAY STATISTICS =====
        
        # Create semi-transparent overlay for stats
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, 0), (350, 300), (0, 0, 0), -1)
        frame = cv2.addWeighted(overlay, 0.7, frame, 0.3, 0)
        
        # Title and performance info
        elapsed_time = time.time() - start_time
        current_fps = frame_count / elapsed_time if elapsed_time > 0 else 0
        
        cv2.putText(frame, "YOLOv11 OBJECT DETECTION", (10, 25),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        cv2.putText(frame, f"FPS: {current_fps:.1f}", (10, 55),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        
        cv2.putText(frame, f"Frame: {frame_count}", (10, 80),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
        
        cv2.putText(frame, f"Confidence: {CONFIDENCE_THRESHOLD}", (10, 100),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
        
        # Object counts header
        cv2.putText(frame, "OBJECT COUNTS (MAX):", (10, 130),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        
        # Display counts for each target object
        y_offset = 155
        
        for display_name, color in TARGET_ITEMS.values():
            count = max_counts[display_name]
            current_count = current_counts[display_name]
            
            # Highlight items detected in current frame
            if current_count > 0:
                status = f" ({current_count} now)"
                text_color = color
            else:
                status = ""
                text_color = (150, 150, 150)
            
            text = f"{display_name}: {count}{status}"
            cv2.putText(frame, text, (20, y_offset),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 2)
            y_offset += 25
        
        # Current vs Max totals
        current_total = sum(current_counts.values())
        max_total = sum(max_counts.values())
        
        cv2.putText(frame, f"Current Frame: {current_total} objects", 
                   (10, y_offset + 10), cv2.FONT_HERSHEY_SIMPLEX, 
                   0.6, (200, 200, 0), 2)
        
        cv2.putText(frame, f"Max Total: {max_total} objects", 
                   (10, y_offset + 35), cv2.FONT_HERSHEY_SIMPLEX, 
                   0.6, (255, 255, 0), 2)
        
        # Controls info
        cv2.putText(frame, "Q: Quit  |  R: Reset  |  S: Save", 
                   (width - 300, height - 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 100, 255), 1)
        
        # ===== DISPLAY FRAME =====
        cv2.imshow("YOLOv11 Real-time Object Detection", frame)
        
        # ===== KEYBOARD CONTROLS =====
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q') or key == 27:  # Q or ESC to quit
            print("\n[INFO] Quitting...")
            break
        elif key == ord('r'):  # R to reset counters
            max_counts = {display_name: 0 for display_name, _ in TARGET_ITEMS.values()}
            print("[INFO] Counters reset")
        elif key == ord('s'):  # S to save screenshot
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            filename = f"detection_{timestamp}.jpg"
            cv2.imwrite(filename, frame)
            print(f"[INFO] Screenshot saved: {filename}")
        elif key == ord('+'):  # + to increase confidence threshold
            CONFIDENCE_THRESHOLD = min(0.9, CONFIDENCE_THRESHOLD + 0.05)
            print(f"[INFO] Confidence threshold increased to: {CONFIDENCE_THRESHOLD}")
        elif key == ord('-'):  # - to decrease confidence threshold
            CONFIDENCE_THRESHOLD = max(0.1, CONFIDENCE_THRESHOLD - 0.05)
            print(f"[INFO] Confidence threshold decreased to: {CONFIDENCE_THRESHOLD}")
    
    # ========== CLEANUP ==========
    cap.release()
    cv2.destroyAllWindows()
    
    # ========== FINAL RESULTS ==========
    print("\n" + "=" * 60)
    print("FINAL DETECTION RESULTS")
    print("=" * 60)
    
    total_detected = 0
    for item, count in max_counts.items():
        if count > 0:
            print(f"{item}: {count}")
            total_detected += count
    
    if total_detected == 0:
        print("[INFO] No target objects detected in the video")
        print("[TIP] Try:")
        print("  1. Lowering confidence threshold (press '-' during detection)")
        print("  2. Using different target objects")
        print("  3. Checking if objects are visible in the video")
    else:
        print("-" * 40)
        print(f"TOTAL OBJECTS DETECTED: {total_detected}")
    
    print(f"\nProcessed {frame_count} frames in {elapsed_time:.1f} seconds")
    print(f"Average FPS: {frame_count/elapsed_time:.1f}" if elapsed_time > 0 else "")
    print("=" * 60)

if __name__ == "__main__":
    main()

YOLOv11 OBJECT DETECTION SYSTEM

[INFO] Loading YOLOv11 model...
[SUCCESS] Model loaded: yolo11n.pt
[INFO] Available items in model: 4/6
  Available: laptop, tv, keyboard, chair
  Not available (need custom training): projector, air conditioner

[INFO] Opening video source...
[INFO] Video Properties:
  • Resolution: 1280x592
  • FPS: 29

[INFO] Starting detection...
[CONTROLS] Press 'Q' to quit | Press 'R' to reset counters
[CONTROLS] Press 'S' to save screenshot

[INFO] Quitting...

FINAL DETECTION RESULTS
Laptop: 1
Monitor: 4
Keyboard: 1
Chair: 4
----------------------------------------
TOTAL OBJECTS DETECTED: 10

Processed 42 frames in 9.6 seconds
Average FPS: 4.4
